In [1]:
import csv

import pandas as pd
import numpy as np

import argparse

In [2]:
file = pd.read_csv('FLIR.csv', sep=';')

In [3]:
file.head()

,path,nombre,temp,csv listo,csv procesado,min,max,inicial,final,rangos +1
0,C:\Users\CCVV\Desktop\10cc55C.csv,10cc55C - flir.csv,"54,4",x,NaN,25,58,68,2800,10
1,C:\Users\CCVV\Desktop\1cc48C.csv,1cc48C - flir.csv,58,x,NaN,26,48,19,2100,10
2,C:\Users\CCVV\Desktop\1cc51C.csv,1cc51C - flir.csv,"59,9",x,NaN,"26,5",51,32,1800,10
3,C:\Users\CCVV\Desktop\1cc55C.csv,1cc55C - flir.csv,"58,6",x,NaN,"27,5",54,82,2200,10
4,C:\Users\CCVV\Desktop\5cc58C.csv,5cc58C - flir.csv,"49,6",x,NaN,24,61,51,3200,10


In [4]:
file['temp']

0    54,4
1      58
2    59,9
3    58,6
4    49,6
5    53,1
Name: temp, dtype: object

In [5]:
for i in file.iterrows():
    print(i)

(0, path             C:\Users\CCVV\Desktop\10cc55C.csv
nombre                          10cc55C - flir.csv
temp                                          54,4
csv listo                                        x
csv procesado                                  NaN
min                                             25
max                                             58
inicial                                         68
final                                         2800
rangos +1                                       10
Name: 0, dtype: object)
(1, path             C:\Users\CCVV\Desktop\1cc48C.csv
nombre                          1cc48C - flir.csv
temp                                           58
csv listo                                       x
csv procesado                                 NaN
min                                            26
max                                            48
inicial                                        19
final                                        2100
rangos +

In [6]:
def processFrame(df, enddata, highLimit, lowLimit, rangosdetemp):
    frameData = []
    maxarea = 220000
    if df.mean().mean() < lowLimit:
        return
    elif df.mean().mean() > highLimit:
        return
    else:
        p100 = sum(df[df>=minimum].count())
        if p100 < maxarea:
            frameData.append(p100)
            for a, b in zip(rangosdetemp, rangosdetemp[1:]):
                frameData.append(sum(df[df>=a][df<b].count()) / p100)
            print(frameData)
            enddata.append(frameData)

In [7]:
def useFile(camino, salida, start, finish, highLimit, lowLimit, rangosdetemp):
    data = []
    with open(camino) as f:
        reader = csv.reader(f, delimiter=';')

        for x in range(finish):
            print('Frame {}'.format(x))
            frame = []
            for _ in range(480):
                frame.append(next(reader))
            if x < start:
                continue
            df = pd.DataFrame(frame)
            df = df.applymap(lambda x: int(float(x.replace(',','.'))))
            processFrame(df, data, highLimit, lowLimit, rangosdetemp)
            
            
    newDF = pd.DataFrame(data)
    newDF.to_csv(salida)

In [40]:
for row in file.iterrows():
    print(row)
    minimum = float(row[1]['min'].replace(',','.'))
    maximum = float(row[1]['max'].replace(',','.'))
    lowLimit = 20
    highLimit = 70
    start = row[1]['inicial']
    rangosdetemp = list(np.linspace(minimum, maximum, num=10))
    finish = row[1]['final']
    useFile(row[1]['path'], row[1]['nombre'], start, 
            finish, highLimit, lowLimit, rangosdetemp)
    break

(0, path             C:\Users\CCVV\Desktop\10cc55C.csv
nombre                          10cc55C - flir.csv
temp                                          54,4
csv listo                                        x
csv procesado                                  NaN
min                                             25
max                                             58
inicial                                         68
final                                         2800
rangos +1                                       10
Name: 0, dtype: object)


AttributeError: 'int' object has no attribute 'replace'

In [56]:

def returnValues(temperature):
    n = 0.0046 * temperature + 0.1449
    k = -3.8484 * temperature + 255.16
    tau = -1.0362 * temperature + 84.246

    return (n, k, tau)

